In [0]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw  

Cloning into 'fenwicks'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 198 (delta 96), reused 146 (delta 51), pack-reused 0
Receiving objects: 100% (198/198), 57.24 KiB | 2.49 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [0]:
BUCKET = 'gs://gs_colab' #@param {type:"string"}
PROJECT_NAME = 'dvc' #@param {type:"string"}
MODEL_NAME = 'InceptionResNetV2' #@param {type:"string"}

BATCH_SIZE = 128 #@param {type: "integer"}
EPOCHS = 20 #@param {type:"integer"}


In [4]:
fw.colab_tpu.setup_gcs()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
data_dir, work_dir, model_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT_NAME, MODEL_NAME)

In [0]:
ws_dir, ws_vars = fw.keras_models.InceptionResNetV2_weights(model_dir)

In [0]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.DVC, os.path.join('.', PROJECT_NAME))
train_dir_local = os.path.join(data_dir_local, 'dogscats/train')
valid_dir_local = os.path.join(data_dir_local, 'dogscats/valid')

In [0]:
train_fn = os.path.join(data_dir, 'train.tfrec')
valid_fn = os.path.join(data_dir, 'valid.tfrec')
_ = fw.io.data_dir_tfrecord(train_dir_local, train_fn)
_ = fw.io.data_dir_tfrecord(valid_dir_local, valid_fn)

In [0]:
class TransferLearningNet(tf.keras.Model):
  def __init__(self, c=256):
    super().__init__()
    self.ir2 = fw.keras_models.get_InceptionResNetV2()
    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(c, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization()
    self.linear = tf.keras.layers.Dense(2, use_bias=False)

  def call(self, x):
    return self.linear(tf.nn.relu(self.bn(self.dense(self.flatten(self.ir2(x))))))

In [0]:
total_steps = 23000 // BATCH_SIZE * EPOCHS

In [0]:
opt_func = fw.train.adam_sgdr_one_cycle(total_steps)
model_func = fw.tpuest.get_clf_model_func(TransferLearningNet, opt_func)

In [0]:
img_size = fw.keras_models.InceptionResNetV2_size()

parser_train = fw.io.get_tfexample_image_parser(img_size, img_size, augment=True)
parser_eval = fw.io.get_tfexample_image_parser(img_size, img_size)

train_input_func = lambda params: fw.io.tfrecord_ds(train_fn, parser_train,
  params['batch_size'], training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(valid_fn, parser_eval, 
  params['batch_size'], training=False)

In [13]:
est = fw.tpuest.get_tpu_estimator(23000, 2000, model_func, work_dir, 
                             ws_dir, ws_vars, BATCH_SIZE)

est.train(train_input_func, steps=total_steps)
est.evaluate(input_fn=valid_input_func, steps=1)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/dvc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.125.222.114:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd99d5b9908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.125.222.114:8470', '_evaluation_master': 'grpc://10.125.222.114:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=179, num_shards=None, num_cores_per_rep

{'accuracy': 0.99, 'global_step': 3580, 'loss': 0.012250643}

In [0]:
fw.io.create_clean_dir(work_dir)